<table align="left">
  <tr>
    <td align="left"><h1>2 : Secure Code Execution</h1></td>
  </tr>
</table>


## Show behavior of custom python interpreter

In [1]:
#!pip install git+https://github.com/huggingface/smolagents.git

In [2]:
from smolagents.local_python_executor import LocalPythonExecutor

custom_executor = LocalPythonExecutor(["numpy"])

In [3]:
def run_capture_exception(command: str):
    try:
        custom_executor(harmful_command)
    except Exception as e:
        print("ERROR:\n", e)

In [4]:
# Example 1: non-defined command
# In Jupyter it works
!echo Bad command

Bad command


In [5]:
# In our interpreter, it does not.
harmful_command="!echo Bad command"
run_capture_exception(harmful_command)

ERROR:
 Code parsing failed on line 1 due to: SyntaxError
!echo Bad command
 ^
Error: invalid syntax (<unknown>, line 1)


In [6]:
[
    're',
    'queue',
    'random',
    'statistics',
    'unicodedata',
    'itertools',
    'math',
    'stat',
    'time',
    'datetime',
    'collections',
    'numpy'
]

['re',
 'queue',
 'random',
 'statistics',
 'unicodedata',
 'itertools',
 'math',
 'stat',
 'time',
 'datetime',
 'collections',
 'numpy']

In [7]:
# Example 2: os not imported
harmful_command="""
import os
exit_code = os.system("echo Bad command")
"""
run_capture_exception(harmful_command)

ERROR:
 Code execution failed at line 'import os' due to: InterpreterError: Import of os is not allowed. Authorized imports are: ['numpy', 'datetime', 'statistics', 'time', 'queue', 'collections', 'unicodedata', 'math', 'itertools', 're', 'stat', 'random']


In [8]:
# Example 3: random._os.system not imported
harmful_command="""
import random
random._os.system('echo Bad command')
"""
run_capture_exception(harmful_command)

ERROR:
 Code execution failed at line 'random._os.system('echo Bad command')' due to: InterpreterError: Forbidden access to module: os


In [9]:
# Example 4: infinite loop
harmful_command="""
while True:
    pass
"""
run_capture_exception(harmful_command)

ERROR:
 Code execution failed at line 'while True:
    pass' due to: InterpreterError: Maximum number of 1000000 iterations in While loop exceeded


In [10]:
custom_executor = LocalPythonExecutor(["PIL"])

harmful_command="""
from PIL import Image

img = Image.new('RGB', (100, 100), color='blue')

i=0
while i < 10000:
    img.save('simple_image_{i}.png')
    i += 1
"""
# custom_executor(harmful_command)
# Let's not execute this but it would not error out, and it would bloat your system with images.

## Running in a sandbox

In [11]:
import os
from dotenv import load_dotenv
load_dotenv()

E2B_API_KEY = os.getenv("E2B_API_KEY")

In [12]:
from smolagents import CodeAgent, HfApiModel, Tool

model = HfApiModel()

class VisitWebpageTool(Tool):
    name = "visit_webpage"
    description = (
        "Visits a webpage at the given url and reads its content as a markdown string. Use this to browse webpages."
    )
    inputs = {
        "url": {
            "type": "string",
            "description": "The url of the webpage to visit.",
        }
    }
    output_type = "string"

    def __init__(self, max_output_length: int = 40000):
        super().__init__()
        self.max_output_length = max_output_length

    def forward(self, url: str) -> str:
        try:
            import re

            import requests
            from markdownify import markdownify
            from requests.exceptions import RequestException

            from smolagents.utils import truncate_content
        except ImportError as e:
            raise ImportError(
                "You must install packages `markdownify` and `requests` to run this tool: for instance run `pip install markdownify requests`."
            ) from e
        try:
            response = requests.get(url, timeout=20)
            response.raise_for_status()  # Raise an exception for bad status codes
            markdown_content = markdownify(response.text).strip()
            markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)
            return truncate_content(markdown_content, self.max_output_length)

        except requests.exceptions.Timeout:
            return "The request timed out. Please try again later or check the URL."
        except RequestException as e:
            return f"Error fetching the webpage: {str(e)}"
        except Exception as e:
            return f"An unexpected error occurred: {str(e)}"

agent = CodeAgent(
    tools=[VisitWebpageTool()],
    model=model,
    executor_type="e2b",
    executor_kwargs={"api_key": E2B_API_KEY},
    max_steps=5
)

Initializing executor, hold on...

Collecting smolagents

  Downloading smolagents-1.14.0-py3-none-any.whl (114 kB)
[?25l     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/114.5 kB[0m [31m?[0m eta [36m-:--:--[0m
[2K     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m114.5/114.5 kB[0m [31m3.9 MB/s[0m eta [36m0:00:00[0m
[?25h
Collecting markdownify>=0.14.1
  Downloading markdownify-1.1.0-py3-none-any.whl (13 kB)

Collecting requests>=2.32.3
  Downloading requests-2.32.3-py3-none-any.whl (64 kB)
[?25l     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/64.9 kB[0m [31m?[0m eta [36m-:--:--[0m[2K
[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m64.9/64.9 kB[0m [31m8.5 MB/s[0m eta [36m0:00:00[0m
[?25hRequirement already satisfied: jinja2>=3.1.4 in /usr/local/lib/python3.10/site-packages (from smolagents) 
(3.1.6)

Collecting python-dotenv
  Downloading python_dotenv-1.1.0-py3-none-any.whl (20 kB)

Collecting rich>=13.9.4
  Downloading rich-14.0.0-py3-none-any.whl (243 kB)
[?25l     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/243.2 kB[0m [31m?[0m eta [36m-:--:--[0m
[2K     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m243.2/243.2 kB[0m [31m9.8 MB/s[0m eta [36m0:00:00[0m
[?25h
Collecting huggingface-hub>=0.28.0
  Downloading huggingface_hub-0.30.2-py3-none-any.whl (481 kB)
[?25l     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/481.4 kB[0m [31m?[0m eta [36m-:--:--[0m
[2K     [91m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m[91m╸[0m [32m481.3/481.4 kB[0m [31m14.5 MB/s[0m eta 
[36m0:00:01[0m[2K     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m481.4/481.4 kB[0m [31m11.3 MB/s[0m eta
[36m0:00:00[0m
[?25h
Collecting pillow>=11.0.0
  Downloading pillow-11.2.1-cp310-cp310-manylinux_2_28_x86_64.whl (4.6 MB)
[?25l     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/4.6 MB[0m [31m?[0m eta [36m-:--:--[0m
[2K     [91m━━━━━[0m[90m╺[0m[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.6/4.6 MB[0m [31m18.9 MB/s[0m eta
[36m0:00:01[0m
[2K     [91m━━━━━━━━━━━━[0m[90m╺[0m[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m1.4/4.6 MB[0m [31m20.8 MB/s[0m eta
[36m0:00:01[0m[2K     [91m━━━━━━━━━━━━━━━━━━━━━[0m[90m╺[0m[90m━━━━━━━━━━━━━━━━━━[0m [32m2.5/4.6 MB[0m [31m23.5
MB/s[0m eta [36m0:00:01[0m
[2K     [91m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m[91m╸[0m[90m━━━━━━━[0m [32m3.8/4.6 MB[0m [31m26.9 MB/s[0m eta
[36m0:00:01[0m
[2K     [91m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m[91m╸[0m [32m4.6/4.6 MB[0m [31m28.5 MB/s[0m eta 
[36m0:00:01[0m[2K     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m4.6/4.6 MB[0m [31m24.0 MB/s[0m eta
[36m0:00:00[0m
[?25h
Collecting duckduckgo-search>=6.3.7

  Downloading duckduckgo_search-8.0.1-py3-none-any.whl (18 kB)

Collecting primp>=0.15.0
  Downloading primp-0.15.0-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
[?25l     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/3.3 MB[0m [31m?[0m eta [36m-:--:--[0m
[2K     [91m━━━━━━━━━━━━━━━━━━━━━━[0m[91m╸[0m[90m━━━━━━━━━━━━━━━━━[0m [32m1.9/3.3 MB[0m [31m55.7 MB/s[0m eta
[36m0:00:01[0m
[2K     [91m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m[91m╸[0m [32m3.3/3.3 MB[0m [31m51.8 MB/s[0m eta 
[36m0:00:01[0m[2K     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m3.3/3.3 MB[0m [31m31.1 MB/s[0m eta
[36m0:00:00[0m
[?25h
Requirement already satisfied: lxml>=5.3.0 in /usr/local/lib/python3.10/site-packages (from 
duckduckgo-search>=6.3.7->smolagents) (5.3.1)
Requirement already satisfied: click>=8.1.8 in /usr/local/lib/python3.10/site-packages (from 
duckduckgo-search>=6.3.7->smolagents) (8.1.8)

Collecting filelock
  Downloading filelock-3.18.0-py3-none-any.whl (16 kB)
Requirement already satisfied: packaging>=20.9 in /usr/local/lib/python3.10/site-packages (from 
huggingface-hub>=0.28.0->smolagents) (24.2)

Collecting fsspec>=2023.5.0
  Downloading f

E2B is running

<div style="font-size: 14px; background-color: #fff9b0; padding: 12px; border-left: 4px solid #facc15;">
  <strong>Note:</strong> The agent is not always successful in this run.
</div>


In [13]:
output = agent.run(
    "Give me one of the top github repos from organization huggingface."
)
print("E2B executor result:", output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me one of the top github repos from organization huggingface.                                              │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

Requirement already satisfied: requests in /usr/local/lib/python3.10/site-packages (2.32.3)
Requirement already satisfied: markdownify in /usr/local/lib/python3.10/site-packages (1.1.0)

Requirement already satisfied: charset-normalizer<4,>=2 in /usr/local/lib/python3.10/site-packages (from requests) 
(3.4.1)
Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.10/site-packages (from requests) 
(2025.1.31)
Requirement already satisfied: urllib3<3,>=1.21.1 in /usr/local/lib/python3.10/site-packages (from requests) 
(1.26.19)
Requirement already satisfied: idna<4,>=2.5 in /usr/local/lib/python3.10/site-packages (from requests) (3.10)
Requirement already satisfied: six<2,>=1.15 in /usr/local/lib/python3.10/site-packages (from markdownify) (1.17.0)
Requirement already satisfied: beautifulsoup4<5,>=4.9 in /usr/local/lib/python3.10/site-packages (from markdownify)
(4.12.3)
Requirement already satisfied: soupsieve>1.2 in /usr/local/lib/python3.10/site-packages (from 
beautifulsoup4<5,>=4.9->markdownify) (2.6)

[33mWARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the 
system package manager. It is recommended to use a virtual environment instead: 
https://pip.pypa.io/warnings/venv[0m[33m
[0m
[1m[[0m[34;49mnotice[0m[1;39;49m][0m[39;49m A new release of pip is available: [0m[31;49m23.0.1[0m[39;49m ->
[0m[32;49m25.1[0m
[1m[[0m[34;49mnotice[0m[1;39;49m][0m[39;49m To update, run: [0m[32;49mpip install --upgrade pip[0m

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  url = "https://github.com/huggingface"                                                                           
  page_content = visit_webpage(url=url)                                                                            
  print(page_content)                                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Hugging Face · GitHub

[Skip to content](#start-of-content)

Navigation Menu
---------------

Toggle navigation

[Sign in](/login?return_to=https%3A%2F%2Fgithub.com%2Fhuggingface)

[huggingface](/huggingface)

* Product

  + [GitHub Copilot

    Write better code with AI](https://github.com/features/copilot)
  + [GitHub Advanced Security

    Find and fix vulnerabilities](https://github.com/security/advanced-security)
  + [Actions

    Automate any workflow](https://github.com/features/actions)
  + [Codespaces

    Instant dev environments](https://github.com/features/codespaces)
  + [Issues

    Plan and track work](https://github.com/features/issues)
  + [Code Review

    Manage code changes](https://github.com/features/code-review)
  + [Discussions

    Collaborate outside of code](https://github.com/features/discussions)
  + [Code Search

    Find more, search less](https://github.com/features/code-search)

  Explore
  + [Why GitHub](https://github.com/why-github)
  + [All features](https://github.com/features)
  + [Documentation](https://docs.github.com)
  + [GitHub Skills](https://skills.github.com)
  + [Blog](https://github.blog)
* Solutions

  By company size
  + [Enterprises](https://github.com/enterprise)
  + [Small and medium teams](https://github.com/team)
  + [Startups](https://github.com/enterprise/startups)
  + [Nonprofits](/solutions/industry/nonprofits)

  By use case
  + [DevSecOps](/solutions/use-case/devsecops)
  + [DevOps](/solutions/use-case/devops)
  + [CI/CD](/solutions/use-case/ci-cd)
  + [View all use cases](/solutions/use-case)

  By industry
  + [Healthcare](/solutions/industry/healthcare)
  + [Financial services](/solutions/industry/financial-services)
  + [Manufacturing](/solutions/industry/manufacturing)
  + [Government](/solutions/industry/government)
  + [View all industries](/solutions/industry)

  [View all solutions](/solutions)
* Resources

  Topics
  + [AI](/resources/articles/ai)
  + [DevOps](/resources/articles/devops)
  + [Security](/resources/articles/security)
  + [Software Development](/resources/articles/software-development)
  + [View all](/resources/articles)

  Explore
  + [Learning Pathways](https://resources.github.com/learn/pathways)
  + [Events & Webinars](https://resources.github.com)
  + [Ebooks & Whitepapers](https://github.com/resources/whitepapers)
  + [Customer Stories](https://github.com/customer-stories)
  + [Partners](https://partner.github.com)
  + [Executive Insights](https://github.com/solutions/executive-insights)
* Open Source

  + [GitHub Sponsors

    Fund open source developers](/sponsors)

  + [The ReadME Project

    GitHub community articles](https://github.com/readme)

  Repositories
  + [Topics](https://github.com/topics)
  + [Trending](https://github.com/trending)
  + [Collections](https://github.com/collections)
* Enterprise

  + [Enterprise platform

    AI-powered developer platform](/enterprise)

  Available add-ons
  + [GitHub Advanced Security

    Enterprise-grade security features](https://github.com/security/advanced-security)
  + [Copilot for business

    Enterprise-grade AI features](/features/copilot/copilot-business)
  + [Premium Support

    Enterprise-grade 24/7 support](/premium-support)
* [Pricing](https://github.com/pricing)

Search or jump to...

Search code, repositories, users, issues, pull requests...
==========================================================

Search

Clear

[Search syntax 
tips](https://docs.github.com/search-github/github-code-search/understanding-github-code-search-syntax)

Provide feedback
================

We read every piece of feedback, and take your input very seriously.

Include my email address so I can be contacted

Cancel
 Submit feedback

Saved searches
==============

Use saved searches to filter your results more quickly
------------------------------------------------------

Name

Query

To see all available qualifiers, see our 
[documentation](https://docs.github.com/search-github/

[Step 1: Duration 0.99 seconds| Input tokens: 2,087 | Output tokens: 112]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
404 Client Error: Not Found for url: 
http://jupyter-api-proxy.internal.dlai/rev-proxy/huggingface_inference_provider/hf-inference/models/Qwen/Qwen2.5-Co
der-32B-Instruct/v1/chat/completions (Request ID: 
Self=1-68105dfa-43799368209b5579330e7383;Root=1-68105dfa-3fc4b923392298261e7138a5;17c51129-7804-4c02-b419-3ceb63d6c
ffb)

[Step 2: Duration 0.16 seconds| Input tokens: 4,174 | Output tokens: 224]

AgentGenerationError: Error in generating model output:
404 Client Error: Not Found for url: http://jupyter-api-proxy.internal.dlai/rev-proxy/huggingface_inference_provider/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions (Request ID: Self=1-68105dfa-43799368209b5579330e7383;Root=1-68105dfa-3fc4b923392298261e7138a5;17c51129-7804-4c02-b419-3ceb63d6cffb)